In [2]:
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import math
from scipy.ndimage import filters
import sys
from scipy.misc import imread
import os
import sklearn.preprocessing

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
from emd import emd
# from https://github.com/garydoranjr/pyemd

ImportError: No module named 'emd'

In [15]:
def readData(filename):
    f = open(filename)
    output = []
    for line in f:
        output.append([float(num) for num in line.strip().split()])
    f.close()
    output = np.array(output)
    return output
        

def getFilenames(filepath):
    '''input: the filepath of the folder of interest
    output: list of all the iflenames in the path of interest'''
    f = []
    for (dirpath, dirnames, filenames) in os.walk(mypath):
        f.extend(filenames)
        break
    return f

def plotFourierRow(row):
    #num observations i think
    N = len(row)
    
    # sample spacing
    ###???? what is this? time, right? time between observations?
    T = 93.0/129
    ###????
    
    x = np.linspace(0.0, N*T, N)
    y = np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
    yf = row
    xf = np.linspace(0.0, 1.0/(2.0*T), N/2)

    plt.plot(xf, 2.0/N * np.abs(yf[0:N/2]))
    plt.grid()
    plt.show()

def EMDist(a, b):
    '''returns earth movers distacne between two numpy arrays'''
    # Initialize a and b numpy arrays with coordinates and weights
    newA = np.zeros((len(a),2))
    newB = np.zeros((len(b),2))

    newA[:,0] = a[:]
    newB[:,0] = b[:]

    for i in range(0,len(a)):
        newA[i][1] = i+1
    for i in range(0,len(b)):
        newB[i][1] = i+1
    # Convert from numpy array to CV_32FC1 Mat
#     a64 = cv2.fromarray(newA)
#     a32 = cv2.CreateMat(a64.rows, a64.cols, cv2.CV_32FC1)
#     cv2.Convert(a64, a32)

#     b64 = cv2.fromarray(newB)
#     b32 = cv2.CreateMat(b64.rows, b64.cols, cv2.CV_32FC1)
#     cv2.Convert(b64, b32)


    # Calculate Earth Mover's
#     return cv2.CalcEMD2(a32,b32,cv2.CV_DIST_L2)
    return emd(newA,newB)



def agglom(data, metric, num_clusters):
    model = AgglomerativeClustering(n_clusters=num_clusters,
                                        linkage="average", affinity=metric)
    model.fit(data)
    plt.figure()
    plt.axes([0, 0, 1, 1])
    for l, c in zip(np.arange(model.n_clusters), 'rgbk'):
        plt.plot(X[model.labels_ == l].T, alpha=.5)
    plt.axis('tight')
    plt.axis('off')
    plt.suptitle("AgglomerativeClustering", size=20)
    plt.show()

    
data = readData('ts_data.txt')
fTransformed = np.array([np.fft.fft(row) for row in data])
data_dists = pairwise_distances(data, metric=EMDist, n_jobs=1)
agglom(data, data_dists, 6)

# print data.shape, fTransformed.shape

KeyboardInterrupt: 

In [14]:
a = np.array([[2,1],
             [5,2],
             [8,3]])
b = np.array([[8,1],
             [9,2],
             [6,3]])
a[:,1]

array([1, 2, 3])

In [71]:
# Input: X Matrix (each row is a training point)
# Output: A vector of labels, corresponding to row indices

np.random.seed(341)

data = scale(data) #####################################

n_samples, n_features = data.shape
print(n_samples, n_features)
#print(data)


k = 15 #expected clusters
sample_size = n_samples
labels = range(n_samples) #If we want to manually label?

print("clusters: %d, \t n_samples %d, \t n_features %d"
      % (k, n_samples, n_features))

print(79 * '_')
print('% 9s' % 'init'
      '    time  inertia    homo   compl  v-meas     ARI AMI  silhouette')

def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))


bench_k_means(KMeans(init='k-means++', n_clusters=k, n_init=10),
              name="k-means++", data=data)

bench_k_means(KMeans(init='random', n_clusters=k, n_init=10),
              name="random", data=data)

# in this case the seeding of the centers is deterministic, hence we run the
# kmeans algorithm only once with n_init=1
pca = PCA(n_components=k).fit(data)
bench_k_means(KMeans(init=pca.components_, n_clusters=k, n_init=7),
              name="PCA-based",
              data=data)
print(79 * '_')

###############################################################################
# Visualize the results on PCA-reduced data

reduced_data = PCA(n_components=2).fit_transform(data)
kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10)
kmeans.fit(reduced_data)

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering on the scaled time series dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

(833L, 129L)
clusters: 15, 	 n_samples 833, 	 n_features 129
_______________________________________________________________________________
init    time  inertia    homo   compl  v-meas     ARI AMI  silhouette
k-means++   0.23s    20101   0.361   1.000   0.531   0.000   -0.000    0.117
   random   0.25s    20373   0.357   1.000   0.526   0.000   -0.000    0.131
PCA-based   0.08s    21009   0.316   1.000   0.480   0.000   -0.000    0.188
_______________________________________________________________________________


In [50]:
sklearn.preprocessing.scale([1.0,10.0,5,8,2])


array([-1.22474487,  1.39970842, -0.05832118,  0.81649658, -0.93313895])

In [74]:
def plotFourierRow(row):
    N = 129
    # sample spacing
    T = 93.0/129
    x = np.linspace(0.0, N*T, N)
    y = np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
    yf = row
    xf = np.linspace(0.0, 1.0/(2.0*T), N/2)

    print xf.shape
    plt.plot(xf, 2.0/N * np.abs(yf[0:N/2]))
    plt.grid()
    plt.show()

IndentationError: unexpected indent (<ipython-input-74-d797abfdc8ea>, line 2)

In [76]:
fTransformed[10]

array([  4.71380000e+04   +0.j        ,  -2.99298352e+03-1439.17616883j,
         5.51640132e+01 -691.59348835j,  -2.58225909e+03-1018.65507723j,
         3.53012581e+02  +48.79324107j,  -4.40510500e+03 +890.47422241j,
        -8.95315751e+02+1201.61754793j,  -9.13871981e+00+3115.72633772j,
         1.96201083e+02+1458.89096181j,  -5.12408664e+02+2005.79632626j,
         2.86413842e+03+1143.36684783j,   8.14842279e+02+2232.2014908j ,
         1.67320995e+03-1878.42378996j,   9.79868408e+02+1090.92214915j,
         3.56167693e+01 -726.58398895j,  -6.19781685e+02 +511.56192661j,
         3.62704642e+02 +469.2925829j ,   1.50371194e+03+1882.60848938j,
         4.57289388e+01-1159.41890549j,  -8.86919446e+01-1258.6663582j ,
        -4.00730046e+01 -388.39920924j,  -2.16496438e+03+2144.71968689j,
        -7.80797413e+02 -926.30120297j,   2.65450579e+03 +462.21477214j,
        -9.91093686e+01  +55.26893353j,  -2.37540621e+03-4121.28379693j,
        -1.52851089e+03 +267.61912464j,  -1.5816424